In [21]:
from Levenshtein import distance
import pandas as pd
import numpy as np
import re

In [8]:
names = pd.read_excel('2_answers.xlsx', encoding='UTF-8')
names.head(5)

,en,ch1,ch2
0,Abril,阿布里尔,阿布丽尔
1,Addy,阿迪,阿迪
2,Afra,阿弗拉,阿芙拉
3,Agace,阿加斯,阿加丝
4,Ainsley,安斯利,安斯莉


In [12]:
dists = []
inds = []
for index, dp in names.iterrows():
    ch1 = dp['ch1']
    ch2 = dp['ch2']
    dists.append(distance(ch1, ch2))
    if distance(ch1, ch2) == 0:
        inds.append(index)

In [17]:
names.loc[inds].drop(columns=['ch2']).to_excel('in_fact_1_not_2.xlsx', encoding='UTF-8-sig', index=False)

In [18]:
all_names = pd.read_excel('1_answers.xlsx', encoding='UTF-8')

In [22]:
ch_pa = re.compile(r'^[\u4E00-\u9FA5]+$')
en_pa = re.compile(r'^[a-zA-Z]+$')
all_names.describe()

,en,ch
count,58335,58335
unique,58320,42924
top,Derlanger,莫尔
freq,2,18


In [23]:
for index, dp in all_names.iterrows():
    en = dp['en']
    ch = dp['ch']
    if not re.findall(en_pa, en) and re.findall(ch_pa, ch):
        print(en, ch)

In [26]:
names = names.drop(inds).reset_index(drop=True)

In [28]:
dists = []
inds = []
for index, dp in names.iterrows():
    ch1 = dp['ch1']
    ch2 = dp['ch2']
    dists.append(distance(ch1, ch2))
    if distance(ch1, ch2) == 0:
        inds.append(index)

In [33]:
sum(dists)/len(dists)

1.1826280623608019

In [34]:
names.to_excel('2_answers.xlsx', encoding='UTF-8', index=False)

In [41]:
def find_diff_one_char(ch1, ch2):
    assert len(ch1) == len(ch2)
    for i in range(len(ch1)):
        if not ch1[i] == ch2[i]:
            return [ch1[i], ch2[i]]
    return []

In [42]:
find_diff_one_char('阿布里尔', '阿布丽尔')

['里', '丽']

In [44]:
alts = pd.DataFrame(columns=[1, 2])
exp = []
for index, dp in names.iterrows():
    ch1 = dp['ch1']
    ch2 = dp['ch2']
    if distance(ch1, ch2) == 1:
        if not len(ch1) == len(ch2):
            exp.append(index)
        else:    
            result = find_diff_one_char(ch1, ch2)
            alts.loc[index] = result

In [52]:
names.loc[exp]

,en,ch1,ch2
59,Bedoyere,贝杜瓦耶尔,贝杜瓦耶
123,Dearie,迪尔里,迪里


extreme exception

In [53]:
names = names.drop(exp)

In [54]:
for index, dp in names.iterrows():
    ch1 = dp['ch1']
    ch2 = dp['ch2']
    if distance(ch1, ch2) == 1: 
        result = find_diff_one_char(ch1, ch2)
        alts.loc[index] = result

In [66]:
alts.describe()

,1,2
count,371,371
unique,42,35
top,利,莉
freq,52,50


In [69]:
alts = alts[alts.duplicated() == False].reset_index(drop=True)

In [72]:
alts

,1,2
0,里,丽
1,弗,芙
2,斯,丝
3,利,莉
4,纳,娜
5,萨,莎
6,利,丽
7,亚,娅
8,玛,马
9,林,琳


In [75]:
def find_diff_chars(ch1, ch2):
    assert len(ch1) == len(ch2)
    result = []
    for i in range(len(ch1)):
        if not ch1[i] == ch2[i]:
            result.append((ch1[i], ch2[i]))
    return result

In [77]:
alts2 = pd.DataFrame()
results = []
for index, dp in names.iterrows():
    ch1 = dp['ch1']
    ch2 = dp['ch2']
    if distance(ch1, ch2) == 2: 
        if not len(ch1) == len(ch2):
            print(ch1, ch2)
        else:
            results.append(find_diff_chars(ch1, ch2))

弗赖伊 弗里


In [79]:
all(len(w) == 2 for w in results)

True

In [89]:
alts2 = pd.DataFrame(columns = [1, 2])
count = 0
for r in results:
    alts2.loc[count] = [r[0][0], r[0][1]]
    alts2.loc[count+1] = [r[1][0], r[1][1]]
    count += 2
    

In [92]:
alts2 = alts2[alts2.duplicated() == False].reset_index(drop=True)

In [97]:
alts = alts.append(alts2, ignore_index=False)

In [98]:
alts = alts[alts.duplicated() == False].reset_index(drop=True)

In [101]:
alts.to_excel('raw_tac.xlsx', encoding='UTF-8-sig', index=False)